In [1]:
import numpy as np
import sys
import cv2
import torch as th
import copy
import blobfile as bf
import PIL
import torchvision
sys.path.insert(1, '/home/mint/guided-diffusion/sample_scripts/sample_utils/')
import mani_utils, inference_utils, params_utils, vis_utils
sys.path.insert(1, '/home/mint/guided-diffusion/guided_diffusion/dataloader/')

def load_image(path):
    with bf.BlobFile(path, "rb") as f:
        pil_image = PIL.Image.open(f)
        pil_image.load()
    pil_image = pil_image.convert("RGB")
    return pil_image

def spiralOrder(X, m, n):
        
    rowStart, rowEnd = 0, m - 1
    colStart, colEnd = 0, n - 1
    idx = []
    while rowStart < rowEnd and colStart < colEnd:
        for i in range(colStart, colEnd + 1):
            # print(X[rowStart][i])
            idx.append([rowStart, i])
        rowStart += 1

        for i in range(rowStart, rowEnd + 1):
            # print(X[i][colEnd])
            idx.append([i, colEnd])
        colEnd -= 1

        if rowStart < rowEnd:
            for i in range(colEnd, colStart - 1, -1):
                # print(X[rowEnd][i])
                idx.append([rowEnd, i])
            rowEnd -= 1

        if colStart < colEnd:
            for i in range(rowEnd, rowStart - 1, -1):
                # print(X[i][colStart])
                idx.append([i, colStart])
            colStart += 1
    return idx

cond, _ = params_utils.load_params(path='/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/', params_key=['shape', 'pose', 'exp', 'cam', 'light', 'detail', 'albedo'])


Key=> shape : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-shape-anno.txt
Key=> pose : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-pose-anno.txt
Key=> exp : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-exp-anno.txt
Key=> cam : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-cam-anno.txt
Key=> light : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-light-anno.txt
Key=> detail : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-detail-anno.txt
Key=> albedo : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-albedo-anno.txt


In [2]:
def grid_sh(n_grid, sh=None, s=1):
    pi = np.pi
    constant_factor = np.array([1/np.sqrt(4*pi), ((2*pi)/3)*(np.sqrt(3/(4*pi))), ((2*pi)/3)*(np.sqrt(3/(4*pi))),\
                        ((2*pi)/3)*(np.sqrt(3/(4*pi))), (pi/4)*(3)*(np.sqrt(5/(12*pi))), (pi/4)*(3)*(np.sqrt(5/(12*pi))),\
                        (pi/4)*(3)*(np.sqrt(5/(12*pi))), (pi/4)*(3/2)*(np.sqrt(5/(12*pi))), (pi/4)*(1/2)*(np.sqrt(5/(4*pi)))])
    sh_light = []
    for lx in np.linspace(-s, s, n_grid):
        for ly in np.linspace(s, 0, n_grid):
            l = np.array((lx, ly, 1))
            l = l / np.linalg.norm(l)
            
            if sh is not None:
                tmp_light = sh.copy()
            else:
                tmp_light = np.zeros((1, 9, 3))
            tmp_light[0:1, 0:1, :] /= 2
            tmp_light[0:1, 1:2, :] = l[0]
            tmp_light[0:1, 2:3, :] = l[1]
            tmp_light[0:1, 3:4, :] = l[2]
            sh_light.append(tmp_light)
    sh_light = np.concatenate(sh_light, axis=0)
    return sh_light

sj = copy.deepcopy(cond['66646.jpg'])
sh_light = grid_sh(sh=sj['light'].reshape(-1, 9, 3), n_grid=10)
# sh_light = grid_sh(n_grid=10)
print(sh_light.shape)
print(sh_light[0])
print(sh_light[1])

# tmp = sj['light'].reshape(-1, 9, 3)
# for i in range(9):
#     x = tmp[0, i, 0]
#     y = tmp[0, i, 1]
#     z = tmp[0, i, 2]
#     print(np.sqrt(x**2 + y**2 + z**2))

(100, 9, 3)
[[ 1.749533    1.7542678   1.7611798 ]
 [-0.57735027 -0.57735027 -0.57735027]
 [ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.57735027  0.57735027]
 [ 0.19125786  0.19213402  0.19291942]
 [-0.01516144 -0.00662446 -0.01495419]
 [ 0.31602845  0.32620078  0.32295486]
 [ 0.44409156  0.44076076  0.4491139 ]
 [ 0.04974668  0.04909366  0.05726353]]
[[ 1.749533    1.7542678   1.7611798 ]
 [-0.59867109 -0.59867109 -0.59867109]
 [ 0.53215208  0.53215208  0.53215208]
 [ 0.59867109  0.59867109  0.59867109]
 [ 0.19125786  0.19213402  0.19291942]
 [-0.01516144 -0.00662446 -0.01495419]
 [ 0.31602845  0.32620078  0.32295486]
 [ 0.44409156  0.44076076  0.4491139 ]
 [ 0.04974668  0.04909366  0.05726353]]


In [3]:
#NOTE: Initialize a DECA renderer
sj = copy.deepcopy(cond['66646.jpg'])
sj['raw_image_path'] = ['/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/66646.jpg']
raw_img = load_image(sj['raw_image_path'][0])
raw_img = th.tensor(np.array(raw_img)[None])
raw_img = raw_img.permute(0, 3, 1, 2)
sj['raw_image'] = raw_img

for k in ['pose', 'shape', 'exp', 'cam', 'light', 'albedo', 'detail']:
    sj[k] = th.tensor(sj[k][None])
mask = params_utils.load_flame_mask()
deca_obj = params_utils.init_deca(mask=mask)

n_step = 144
n_grid = int(np.sqrt(n_step))
s = 5
sh_light = grid_sh(sh=sj['light'].reshape(-1, 9, 3).cpu().numpy(), n_grid=int(np.sqrt(n_step)))
# sh_light = grid_sh(n_grid=n_grid, s=s)
sj['light'] = th.tensor(sh_light)
# print(sj['light'])

# sj['R_normals'] = params_utils.get_R_normals(n_step=n_step)
deca_rendered, _ = params_utils.render_deca(deca_params=sj, 
                                            idx=0, n=n_step, 
                                            avg_dict=None, 
                                            render_mode='shape', 
                                            rotate_normals=False, 
                                            mask=mask,
                                            deca_obj=deca_obj)


creating the FLAME Decoder
[#] DECA : SRenderY applied mask


/home/mint/guided-diffusion/sample_scripts/sample_utils/params_utils.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'light':th.tensor(deca_params['light']).to(device).reshape(-1, 9, 3).float(),


In [4]:
out = (deca_rendered.permute(0, 2, 3, 1).mul(255).add_(0.5).clamp_(0, 255)).type(th.ByteTensor)
# torchvision.io.write_video(video_array=out, filename=f"./ren.mp4", fps=30)

import os
path = './face/'
os.makedirs(path, exist_ok=True)
print(out.shape)
B, H, W, C = out.shape
frames = out.reshape(n_grid, n_grid, H, W, C)
for ili, li in enumerate(np.linspace(-s, s, n_grid)):
    for ilj, lj in enumerate(np.linspace(-s, s, n_grid)):
        frame = frames[ili, ilj].cpu().detach()
        frame = frame.permute(2, 0, 1)
        frame = (frame / 255.0).float()
        torchvision.utils.save_image(tensor=(frame), fp=f"{path}/{ilj:02d}_{ili:02d}.png")
    
videos = []
x_grid = np.linspace(-s, s, n_grid)
y_grid = np.linspace(-s, s, n_grid)
spiral = spiralOrder(frames, m=n_grid, n=n_grid)
for idx in spiral:
    videos.append(frames[idx[0], idx[1]])
torchvision.io.write_video(video_array=th.stack(videos, dim=0), filename=f"./ren3.mp4", fps=30)
          

torch.Size([144, 256, 256, 3])
